# Extracting metadata information from a satellite image



In [16]:
# user rasterio to open a 3-band (red, green, blue) PlanetScope visual asset

import rasterio
satdat = rasterio.open("Rome_visual.tif")

In [17]:
# Minimum bounding box in projected units

print(satdat.bounds)

BoundingBox(left=288596.8, bottom=4637965.6, right=294747.2, top=4644686.399999999)


In [18]:
# Get dimensions, in map units (using the example GeoTIFF, that's meters)

width_in_projected_units = satdat.bounds.right - satdat.bounds.left
height_in_projected_units = satdat.bounds.top - satdat.bounds.bottom

print("Width: {}, Height: {}".format(width_in_projected_units, height_in_projected_units))

Width: 6150.400000000023, Height: 6720.799999999814


In [19]:
# Number of rows and columns.

print("Rows: {}, Columns: {}".format(satdat.height, satdat.width))

Rows: 8401, Columns: 7688


In [20]:
# This dataset's projection uses meters as distance units.  What are the dimensions of a single pixel in meters?

xres = (satdat.bounds.right - satdat.bounds.left) / satdat.width
yres = (satdat.bounds.top - satdat.bounds.bottom) / satdat.height

print(xres, yres)
print("Are the pixels square: {}".format(xres == yres))

0.800000000000003 0.7999999999999778
Are the pixels square: False


In [21]:
# Get coordinate reference system

satdat.crs

ERROR 1: PROJ: internal_proj_identify: /Applications/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


CRS.from_wkt('PROJCS["WGS 84 / UTM zone 33N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]')

In [22]:
# Convert pixel coordinates to world coordinates.

# Upper left pixel
row_min = 0
col_min = 0

# Lower right pixel.  Rows and columns are zero indexing.
row_max = satdat.height - 1
col_max = satdat.width - 1

# Transform coordinates with the dataset's affine transformation.
topleft = satdat.transform * (row_min, col_min)
botright = satdat.transform * (row_max, col_max)

print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

Top left corner coordinates: (288596.8, 4644686.399999999)
Bottom right corner coordinates: (295316.8, 4638536.8)


In [23]:
# All of the metadata required to create an image of the same dimensions, datatype, format, etc. is stored in
# the dataset's profile:

satdat.profile

ERROR 1: PROJ: internal_proj_identify: /Applications/anaconda3/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 7688, 'height': 8401, 'count': 4, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 33N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(0.8, 0.0, 288596.8,
       0.0, -0.8, 4644686.399999999), 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}

## File Compression

Raster datasets use **compression** to reduce filesize. There are a number of compression methods, all of which fall into two categories: lossy and lossless. _Lossless_ compression methods retain the original values in each pixel of the raster, while _lossy_ methods result in some values being removed. Because of this, lossy compression is generally not well-suited for analytic purposes, but can be very useful for reducing storage size of visual imagery.

All Planet data products are available as GeoTIFFs using lossless LZW compression. By creating a lossy-compressed copy of a visual asset, we can significantly reduce the dataset's filesize. In this example, we will create a copy using the "JPEG" lossy compression method:

In [24]:
pip install humanize

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
from humanize import naturalsize as sz

# returns size in bytes
size = os.path.getsize("Rome_visual.tif")

# output a human-friendly size
print(sz(size))

258.4 MB


## Copying a dataset 

In [26]:
# read all bands from source dataset into a single 3-dimensional ndarray

data = satdat.read()

In [27]:
# write new file using profile metadata from original dataset
# and specifying JPEG compression

profile = satdat.profile
profile['compress'] = 'JPEG'

with rasterio.open('Rome_compressed.tif', 'w', **profile) as dst:
    dst.write(data)

### Lossy compression results

In [28]:
new_size = os.path.getsize("Rome_compressed.tif")
print(sz(new_size))

67.9 kB
